# Building wind suitablity layers for GRIDCERF

The following code was used to build the wind suitability layers for GRIDCERF. GRIDCERF does not provide the source data directly due to some license restrictions related for direct redistribution of the unaltered source data.  However, the following details the provenance associated with each source dataset and how they were processed.


## 1. Downloading the data

### 1.1 Download GRIDCERF

Download the GRIDCERF package if you have not yet done so from here:  https://doi.org/10.57931/2281697.  Please extract GRIDCERF inside the `data` directory of this repository as the paths in this notebook are set to that expectation.


### 1.2 Download the data


- **Title**: Wind Speed (100m Annual Average)
- **Description from Source**:   Depicts mean annual wind speed at a height of 100 meters, gridded at a spatial resolution of 2,000 meters.
- **Source URL**:  https://gem.anl.gov/tool
- **Date Accessed**:  10/3/23
- **Citation**
> Draxl, C., Clifton, A., Hodge, B.-M. & McCaa, J. The Wind Integration National Dataset (WIND) Toolkit. Applied Energy 151, 355–366 (2015).

- **Application**: The 100m annnual average wind speed data contains raster data for average wind speeds at 10m, 40m, 60m, 80m, 100m, 120m, 140m, 160m, and 200m for both onshore and offshore areas. To align with current and future utility scale wind farms, only 80m and above data is used.

_____

- **Title**: 100-Meter Resolution Elevation of the Conterminous United States
- **Description from Source**:   This layer is a georeferenced raster image displaying elevation data for the conterminous United States. These data were derived from the National Elevation Dataset (NED) released in October, 2012, The elevation data shows the terrain at a resolution of 100 meters. The NED is a raster product assembled by the U.S. Geological Survey, designed to provide national elevation data in a seamless form with a consistent datum, elevation unit, and projection. 
- **Source URL**:  https://earthworks.stanford.edu/catalog/stanford-zz186ss2071
- **Date Accessed**:  10/3/23
- **Citation**
> National Atlas of the United States, 2012. 100-Meter Resolution Elevation of the Conterminous United States. National Atlas of the United States. Available at: http://purl.stanford.edu/zz186ss2071.
- **Application**: To determine which areas are capable of meeting specific capacity factors for wind generation, the wind power density at the wind turbine rotor elevation must be calculated. Wind power density is equal to:

$$ PowerDensity = EfficiencyRating * ρ * RotorArea * WindSpeed^3   \ (1) $$

Where, ρ = density of the air in kg/m^3, EfficiencyRating is the efficiency of the turbine, wind speed is in m/s^2. The RotorArea is the cross-sectional area of the wind in m^2, and the wind speed is in meters per second.

Air density varies based on altitude. To estimate the altitude at each location in the CONUS, the elevation at each point is used in conjunction with the hub height of the turbine being evaluated.

For more information on the power density calculation see:

> Kasper, Dan, 2023. Wind Energy and Power Calculations. PennState College of Earth and Mineral Sciences. Available at: https://www.e-education.psu.edu/emsc297/node/649 [Accessed Oct 3, 2023]

Air density at various elevations are provided at the following reference

> The Engineering ToolBox (2003). U.S. Standard Atmosphere vs. Altitude. [online] Available at: https://www.engineeringtoolbox.com/standard-atmosphere-d_604.html [Accessed Oct 3, 2023].
_____

## 2. Setup environment

### 2.1 Install GDAL

This application requires GDAL to be installed.  We will call GDAL directly from your command prompt or terminal, so please ensure that you can do so before running the following cells.  More information on how to install GDAL can be found here:  https://gdal.org/download.html


### 2.2 Import necessary Python packages

In [1]:
import os
import glob

import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
import math
from rasterio.plot import show

## 3. Configuration

In [20]:
# get the parent directory path to where this notebook is currently stored
root_dir = os.path.dirname(os.getcwd())

# data directory in repository
data_dir = os.path.join(root_dir, "data")

# GRIDCERF data directory from downloaded archive
gridcerf_dir = os.path.join(data_dir, "gridcerf")

# GRIDCERF reference data directory
reference_dir = os.path.join(gridcerf_dir, "reference")

# GRIDCERF common data directory
common_dir = os.path.join(gridcerf_dir, "common")

# GRIDCERF technology_specific data directory
technology_specific_dir = os.path.join(gridcerf_dir, "technology_specific")

# GRIDCERF compiled final suitability data directory
compiled_dir = os.path.join(gridcerf_dir, "compiled")

# template land mask raster
template_raster = os.path.join(reference_dir, "gridcerf_landmask.tif")

# source wind data directory
source_dir = os.path.join(gridcerf_dir, "source", "technology_specific")


wind_capfac_dict = {
    'onshore': [x / 100 for x in list(range(5, 55, 5))],
    'offshore':[x / 100 for x in list(range(25, 65, 5))]}


# create wind development scenarios
wind_dict = {
    'onshore':{
        80:{
            'rotor_diameter_m':125, 
            'turbine_kw':2500, 
            'efficiency_rating':.3},
        100:{
            'rotor_diameter_m':150, 
            'turbine_kw':4000, 
            'efficiency_rating':.3},
        120:{
            'rotor_diameter_m':175, 
            'turbine_kw':5500, 
            'efficiency_rating':.3},
        140:{
            'rotor_diameter_m':200, 
            'turbine_kw':7000, 
            'efficiency_rating':.3}},
    'offshore':{
        100:{
            'rotor_diameter_m':159, 
            'turbine_kw':8000, 
            'efficiency_rating':.4},
        140:{
            'rotor_diameter_m':240, 
            'turbine_kw':15000, 
            'efficiency_rating':.4},
        160:{
            'rotor_diameter_m':263, 
            'turbine_kw':18000, 
            'efficiency_rating':.4}}}

# cut in speed m/s for minimum wind generation
cut_in_speed = 4

## 4. Generate wind suitability rasters

### 4.1 Functions to build suitability

In [3]:
def convert_meters_to_feet(meters):
    
    return meters*3.28084

def calc_air_density(total_altitude_ft):
    
    condition1 = ((total_altitude_ft >= -2000) & (total_altitude_ft < 2000))
    condition2 = ((total_altitude_ft >= 2000) & (total_altitude_ft < 4000))
    condition3 = ((total_altitude_ft >= 4000) & (total_altitude_ft < 6000))
    condition4 = ((total_altitude_ft >= 6000) & (total_altitude_ft < 8000))
    condition5 = ((total_altitude_ft >= 8000) & (total_altitude_ft < 10000))
    condition6 = ((total_altitude_ft >= 10000) & (total_altitude_ft < 12000))
    condition7 = ((total_altitude_ft >= 12000) & (total_altitude_ft < 14000))
    condition8 = ((total_altitude_ft >= 14000) & (total_altitude_ft < 16000))
    
    air_density = np.where(condition1, 1.225, 
                           np.where(condition2, 1.155,
                                   np.where(condition3, 1.088,
                                           np.where(condition4,1.024,
                                                   np.where(condition5, 0.963,
                                                           np.where(condition6, 0.904,
                                                                   np.where(condition7, 0.849,
                                                                           np.where(condition8, 0.796, 0.74))))))))
    
    return air_density

def process_elevation_file():
    """ 
    Process original elevation raster to warp to the correct extent and CRS.
    """
    # set local paths
    elev_source_file = os.path.join(source_dir, "stanford_elevation", "stanford-zz186ss2071-geotiff.tif")
    temp_output_file = os.path.join(source_dir, "temporary_raster.tif")
    
    # warp raster and convert to CRS
    gdal_warp_cmd = f"gdalwarp -te -2831615.228 -1539013.3223 2628318.0948 1690434.1707 -tr 1000.0 1000.0 -t_srs ESRI:102003 -dstnodata 0 -overwrite {elev_source_file} {temp_output_file}"
    os.system(gdal_warp_cmd)
    
    # fill nodata areas with 0
    raster = rasterio.open(temp_output_file)
    elevation = raster.read(1)
    elevation_array = np.where(elevation <=0, 0, elevation)
                                   
    return elevation_array
    
def calculate_altitude(hub_height_m, elevation_array):
    
    # initialize an altitude raster with 0s
    altitude = np.empty(elevation_array.shape, dtype=rasterio.float32)

    # calculate the altitude
    meters_to_feet_calc = np.vectorize(convert_meters_to_feet)

    altitude = meters_to_feet_calc(elevation_array + hub_height_m)
    
    return altitude

def create_air_density_array(altitude_array):
    
    # create an empty array
    air_density_array = np.empty(altitude_array.shape, dtype=rasterio.float32)

    air_density_array = calc_air_density(altitude_array)

    return air_density_array

def calculate_wind_power_density(air_density_array, wind_speed_array, efficiency_rating):
   
    alpha = 0.5
    beta = 3
    wind_power_density_array = efficiency_rating * alpha * air_density_array * (wind_speed_array**beta)
    
    return wind_power_density_array
    

def process_wind_speed_file(hub_height_m):
    
    # set local paths
    wind_source_file = os.path.join(source_dir, "nrel_wind_speed", 'wind_annual_speed_100m_v2', f'wtk_conus_{hub_height_m}m_mean_masked.tif')
    temp_output_file = os.path.join(source_dir, "temporary_raster.tif")

    gdal_warp_cmd = f"gdalwarp -te -2831615.228 -1539013.3223 2628318.0948 1690434.1707 -tr 1000.0 1000.0 -t_srs ESRI:102003 -overwrite {wind_source_file} {temp_output_file}"
    os.system(gdal_warp_cmd)
    
    wind_raster = rasterio.open(temp_output_file)
    wind_speed_array = wind_raster.read(1)
    
    return wind_speed_array

def calculate_rotor_area_m2(rotor_diameter_m):
    
    rotor_area_m2 = math.pi * ((rotor_diameter_m/2)**2)
    
    return rotor_area_m2

def create_generation_array(rotor_area_m2, wind_power_density_array):

    # calculate the potential energy output
    kw_output_array = ((rotor_area_m2 * wind_power_density_array)) / 1000
    kwh_generation_array = kw_output_array * 8760
    
    return kwh_generation_array

def calculate_required_gen_kwh(cf, turbine_kw):
    
    kwh = turbine_kw * 8760 * cf
    
    return kwh

def compile_wind_suit(cf, hub_height_m, rotor_diameter_m, turbine_kw, efficiency_rating):
          
    elevation_array = process_elevation_file()
    altitude_array = calculate_altitude(hub_height_m = hub_height_m, elevation_array=elevation_array)
    
    air_density_array = create_air_density_array(altitude_array)

    # read in reprojected and warped wind speed data file corresponding to hub height
    wind_speed_array = process_wind_speed_file(hub_height_m=hub_height_m)

    # calculate the power density array
    wind_power_density_array = calculate_wind_power_density(air_density_array=air_density_array,
                                                          wind_speed_array=wind_speed_array,
                                                          efficiency_rating=efficiency_rating)
    # calculate rotor area
    rotor_area_m2 = calculate_rotor_area_m2(rotor_diameter_m=rotor_diameter_m)
    
    # calculate the wind generation array
    wind_generation_array = create_generation_array(rotor_area_m2=rotor_area_m2, 
                                                    wind_power_density_array=wind_power_density_array)
    
    required_generation_kwh = calculate_required_gen_kwh(cf=cf, turbine_kw=turbine_kw)
    
    # set array to 1 or 0 based on suitability
    kwh_output_cf_array = np.where(wind_generation_array >= required_generation_kwh, 0, 1)   
    
    return kwh_output_cf_array

### 4.2 Generate wind suitability rasters for all desired hub heights

In [12]:
# read in template for metadata
template = rasterio.open(template_raster)

for wind_type in wind_dict:
    for hub_height in wind_dict[wind_type]:
        
        # collect wind parameters
        rotor_diameter_m = wind_dict[wind_type][hub_height]['rotor_diameter_m']
        turbine_kw = wind_dict[wind_type][hub_height]['turbine_kw']
        efficiency_rating = wind_dict[wind_type][hub_height]['efficiency_rating']
        
        # collect capacity factors
        for cf in wind_capfac_dict[wind_type]:
            
            print(f'\n Processing {wind_type} wind for {hub_height}m and {cf} capacity factor...')
            
            # create output file name and path
            output_tif_file_name = f"gridcerf_nrel_wind_{wind_type}_potential_{hub_height}m_{int(cf*100)}cf.tif"
            output_raster_path = os.path.join(technology_specific_dir, output_tif_file_name)

            # computer potential array
            wind_potential_array = compile_wind_suit(cf=cf, hub_height_m=hub_height, 
                                                     rotor_diameter_m=rotor_diameter_m, 
                                                     turbine_kw=turbine_kw, 
                                                     efficiency_rating=efficiency_rating)
            # collect metadata
            metadata = template.meta.copy()

            # write file
            with rasterio.open(output_raster_path, 'w', **metadata) as dest:
    
                dest.write(wind_potential_array, 1)


 Processing onshore wind for 80m and 0.05 capacity factor...
Copying color table from /Users/mong275/repos/gridcerf/data/gridcerf/source/technology_specific/stanford_elevation/stanford-zz186ss2071-geotiff.tif to new file.
Creating output file that is 5460P x 3229L.
Processing /Users/mong275/repos/gridcerf/data/gridcerf/source/technology_specific/stanford_elevation/stanford-zz186ss2071-geotiff.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Users/mong275/repos/gridcerf/data/gridcerf/source/technology_specific/stanford_elevation/stanford-zz186ss2071-geotiff.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 5460P x 3229L.
Processing /Users/mong275/repos/gridcerf/data/gridcerf/source/technology_specific/nrel_wind_speed/wind_annual_speed_100m_v2/wtk_conus_80m_mean_masked.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Users/mong275/repos/gridcerf/data/gridcerf/source/technology_specific/nrel_wind_speed/wind_annual_spee

### 4.3 Create an optional raster for areas with no wind potential

In [21]:
template = rasterio.open(template_raster)
metadata = template.meta.copy()

for wind_type in wind_dict:
    for hub_height in wind_dict[wind_type]:
    
        # create an array of wind speeds
        wind_speed_array = process_wind_speed_file(hub_height_m=hub_height)

        # change to binary for potential/no potential
        no_wind_potential = np.where(wind_speed_array < cut_in_speed, 1, 0)

        
        output_tif_file_name = f"gridcerf_wind_{wind_type}_no_potential_{hub_height}m.tif"
        output_raster_path = os.path.join(technology_specific_dir, output_tif_file_name)
        with rasterio.open(output_raster_path, 'w', **metadata) as dest:
    
            dest.write(no_wind_potential, 1)

Creating output file that is 5460P x 3229L.
Processing /Users/mong275/repos/gridcerf/data/gridcerf/source/technology_specific/nrel_wind_speed/wind_annual_speed_100m_v2/wtk_conus_80m_mean_masked.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Users/mong275/repos/gridcerf/data/gridcerf/source/technology_specific/nrel_wind_speed/wind_annual_speed_100m_v2/wtk_conus_80m_mean_masked.tif.
Copying nodata values from source /Users/mong275/repos/gridcerf/data/gridcerf/source/technology_specific/nrel_wind_speed/wind_annual_speed_100m_v2/wtk_conus_80m_mean_masked.tif to destination /Users/mong275/repos/gridcerf/data/gridcerf/source/technology_specific/temporary_raster.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 5460P x 3229L.
Processing /Users/mong275/repos/gridcerf/data/gridcerf/source/technology_specific/nrel_wind_speed/wind_annual_speed_100m_v2/wtk_conus_100m_mean_masked.tif [1/1] : 0Using internal nodata values (e.g. 0) for image 